<div align="center">
    <h1>DS-210: Programming for Data Science</h1>
    <h1>Lecture 18</h1>
</div>


# 1. Structs
# 2. Memory Management: Stack and Heap


## Structs

Last time: tuples, e.g., `(12, 1.7, true)`

Structs compared to tuples:<br>
$\bullet$ **Similar:** can hold a few items of different types<br>
$\bullet$ **Different:** the items have names

In [2]:
// Definition: list items (called fields)
//             and their types

struct Person {
    name: String,
    year_born: u16,
    time_100m: f64,
    likes_ice_cream: bool,
}

In [5]:
// Instantiation: replace types with values

let mut cartoon_character: Person = Person {
    name: String::from("Tasmanian Devil"),
    year_born: 1954,
    time_100m: 7.52,
    likes_ice_cream: true,
};


In [6]:
// Accessing fields: use ".field_name"
println!("{} was born in {}",
    cartoon_character.name,
    cartoon_character.year_born);
cartoon_character.year_born = 2022; // Modify the value in struct
println!("{} was born in {}",
    cartoon_character.name,
    cartoon_character.year_born);

Tasmanian Devil was born in 1954
Tasmanian Devil was born in 2022


<br>
<div align="center">
    <b>Structs vs tuples:<br>Which are better?</b>
</div>


## Tuple structs

Named tuples to impose more meaning and delineate a different type.  Notice the different syntax () vs {}.

Example: both `(f64,f64,f64)`
* box size (e.g., 8.5 in${}\times{}$11 in${}\times{}$6 in)
* Euclidean coordinates of a point in 3D

In [7]:
struct BoxSize(f64,f64,f64);
struct Point2(f64,f64,f64);

In [76]:
let mut my_box = BoxSize(3.2,6.0,2.0);
let mut p : Point2 = Point2(-1.3,2.1,0.0);

In [77]:
// won't work
my_box = p;

// Impossible to accidentally confuse different
// types of triples.
// No runtime penalty! Verified at compilation.

Error: mismatched types

In [78]:
// Acessing via index
println!("{} {} {}",p.0,p.1,p.2);
p.0 = 17.2;

// Destructuring
let Point2(first,second,third) = p;
println!("{} {} {}", first, second, third);

-1.3 2.1 0
17.2 2.1 0


## Named structs in enums

Structs with braces and exchangable with tuples in many places

In [79]:
enum LPSolution {
    None,
    Point{x:f64,y:f64}
}

let example = LPSolution::Point{x:1.2, y:4.2};

In [80]:
if let LPSolution::Point{x:first,y:second} = example {
    println!("coordinates: {} {}", first, second);
};

coordinates: 1.2 4.2


### Structs are critical to Rust's OO capabilities

In [81]:
struct Point {
    x: f64,
    y: f64,
}

struct Rectangle {
    p1: Point,
    p2: Point,
}

impl Rectangle {
    // This is a method
    fn area(&self) -> f64 {
        // `self` gives access to the struct fields via the dot operator
        let Point { x: x1, y: y1 } = self.p1;
        let Point { x: x2, y: y2 } = self.p2;

        // `abs` is a `f64` method that returns the absolute value of the
        // caller
        ((x1 - x2) * (y1 - y2)).abs()
    }

    fn perimeter(&self) -> f64 {
        let Point { x: x1, y: y1 } = self.p1;
        let Point { x: x2, y: y2 } = self.p2;

        2.0 * ((x1 - x2).abs() + (y1 - y2).abs())
    }
}

let rectangle = Rectangle {
    p1: Point{x:0.0, y:0.0},
    p2: Point{x:3.0, y:4.0},
};

println!("Rectangle perimeter: {}", rectangle.perimeter());
println!("Rectangle area: {}", rectangle.area());


Rectangle perimeter: 14
Rectangle area: 12


## Memory Management: Stack vs. Heap

* Two different places where space for data can be allocated
* We will discuss them one by one


## Stack

* LIFO (last in first out) memory allocation
* Stores current local variables and additional information such as:
  - function arguments
  - function output
  - where to continue when a function terminates
* Fast memory allocation
* Usually small fraction of the memory
* Often: size of the allocated memory has to be known in advance (compilation time)

Almost everything you saw so far allocated on stack
* Exception: data in `String` allocated on heap

## Stack example (idealized)

In [82]:
fn main() {
    let mut x = 3;
    let mut y = 8;
    println!("x = {}, y = {}",x,y);
    x = add_or_subtract(x,y,true); // x = x + y
    y = add_or_subtract(x,y,false); // y = x - y
    x = add_or_subtract(x,y,false); // x = x - y
    println!("x = {}, y = {}",x,y);
}

fn add_or_subtract(x:i32, y:i32, add:bool) -> i32 {
    let second_arg = if add {y} else {negate(y)};
    x + second_arg
}

fn negate(x:i32) -> i32 {
    -x
}

main();


x = 3, y = 8
x = 8, y = 3


### Step 1: call `main`
* `x` and `y` allocated on stack and initiated
* Stack: `main` (`x`, `y`)

### Step 2: call `add_or_subtract` (1st time)
* arguments for `add_or_subtract` put on stack
* space for solution allocated on stack
* space for `second_arg` allocated as well
* Stack: `main` (`x`, `y`), `add_or_subtract` (all the above + auxiliary information)

## Stack example (idealized)

In [83]:
fn main() {
    let mut x = 3;
    let mut y = 8;
    println!("x = {}, y = {}",x,y);
    x = add_or_subtract(x,y,true);
    y = add_or_subtract(x,y,false);
    x = add_or_subtract(x,y,false);
    println!("x = {}, y = {}",x,y);
}

fn add_or_subtract(x:i32, y:i32, add:bool) -> i32 {
    let second_arg = if add {y} else {negate(y)};
    x + second_arg
}

fn negate(x:i32) -> i32 {
    -x
}

main();


x = 3, y = 8
x = 8, y = 3


### Step 3: `add_or_subtract` terminates
* process and remove all information about the 
call
* Stack: `main` (`x`, `y`)

### Step 4: call `add_or_subtract` (2nd time)
* arguments for `add_or_subtract` put on stack
* space for solution allocated on stack
* space for `second_arg` allocated as well
* Stack: `main` (`x`, `y`), `add_or_subtract` (all the above + auxiliary information)

## Stack example (idealized)

In [84]:
fn main() {
    let mut x = 3;
    let mut y = 8;
    println!("x = {}, y = {}",x,y);
    x = add_or_subtract(x,y,true);
    y = add_or_subtract(x,y,false);
    x = add_or_subtract(x,y,false);
    println!("x = {}, y = {}",x,y);
}

fn add_or_subtract(x:i32, y:i32, add:bool) -> i32 {
    let second_arg = if add {y} else {negate(y)};
    x + second_arg
}

fn negate(x:i32) -> i32 {
    -x
}

main();


x = 3, y = 8
x = 8, y = 3


### Step 5: call `negate` (1st time)
* the argument for `negate` put on stack
* space for solution allocated on stack
* Stack: `main` (`x`, `y`), `add_or_subtract` (...), `negate` (all of the above + auxiliary information)

### Step 6: `negate` terminates
* process and remove all information about the 
call
* Stack: `main` (`x`, `y`), `add_or_subtract` (...)

## Stack example (idealized)

In [39]:
fn main() {
    let mut x = 3;
    let mut y = 8;
    println!("x = {}, y = {}",x,y);
    x = add_or_subtract(x,y,true);
    y = add_or_subtract(x,y,false);
    x = add_or_subtract(x,y,false);
    println!("x = {}, y = {}",x,y);
}

fn add_or_subtract(x:i32, y:i32, add:bool) -> i32 {
    let second_arg = if add {y} else {negate(y)};
    x + second_arg
}

fn negate(x:i32) -> i32 {
    -x
}

main();



x = 3, y = 8
x = 8, y = 3


### Step 7: `add_or_subtract` terminates
* [...]
* Stack: `main` (`x`, `y`)

### Step 8: call `add_or_subtract` (3rd time)
* [...]
* Stack: `main` (`x`, `y`), `add_or_subtract` (...)

<div align="center">
    <h1>...</h1>
</div>

## Limited space on stack!

In [85]:

fn same_number(x:u32) -> u32 {
    match x {
        0 => 0,
        _ => 1 + same_number(x - 1),
    }
}

In [86]:
same_number(7)

7

In [87]:
same_number(123_456)

123456

In [88]:
same_number(1_000_000)

Error: Subprocess terminated with status: signal: 11 (SIGSEGV)

## Using too much memory on stack: *stack overflow*

This is where the name of the popular webpage for asking questions about programming comes from!<br>

<div align="center">
    <img src="chucknorris.png" alt="[screenshot of stackoverflow.com]">
</div>

## Heap

* Memory allocated and freed in arbitrary order
* Arbitrary amount allocated
* The application knows a *pointer*${}={}$the address of assigned memory

<div align="center">
<b>Pros and cons?</b>
</div>

Pros:
* Arbitrary amount of data
* No copying to pass data around
  * Just share the pointer!


Cons:
* Slower allocation:
  * Possible request for more space to the operating system
* Possible memory fragmentation
* Slower access:
  * Have to follow the pointer to get to data


## Stack vs. heap in Python

* Elementary pieces of data allocated on stack: integers, floats, Boolean values, ...

* Anything else allocated on the heap

<br><br>
<div align="center">
    <h2>[Switch to the Python notebook]</h2>
</div>

## Bonus content: stack overflow?

In [89]:
fn same_number_2(x:u64) -> u64 {
    fn same_number_aux(y:u64, accumulate:u64) -> u64 {
        match y {
            0 => accumulate,
            _ => same_number_aux(
                y - 1,
                accumulate + 1),
        }
    }
    return same_number_aux(x,0);
}

In [90]:
same_number_2(1234)

1234

In [91]:
same_number_2(1_000_000_000)

1000000000

### Memory ownership semantics

In [3]:
let mut s = String::from("hello");
s.push_str(", world!"); 
println!("{}", s);
let s2 = s;
println!("{}", s2);
let s3 = s;

Error: use of moved value: `s`

$\bullet$ **No stack overflow!** Why? Look up **tail call** and **tail recursion**.<br>
$\bullet$ Not guaranteed in Rust, but sometimes works.